In [1]:
import importlib
import sys
import numpy as np
from MeasurementInfo import MeasurementInfo #class for inputting measurement info
from RunInfo import RunInfo #class for inputting run info. Also contains most of the useful code
import heapq
from scipy import signal
from scipy.optimize import curve_fit
import AnalyzePDE
from AnalyzePDE import SPE_data
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib as mpl
import ProcessWaveforms_MultiGaussian
from ProcessWaveforms_MultiGaussian import WaveformProcessor as WaveformProcessor
#from AnalyzePDE import Alpha_data

## 192K data first

In [ ]:
proms = [0.0018,0.002,0.0033,0.0033,0.0035]#0.002,0.00088,0.0026,0.0026,0.0035
upperlim = [0.35,0.35,0.35,0.35,0.35]

In [2]:
###Post IV Data 192 K
run_spe_solicited = RunInfo(['Run_1658220019.hdf5'], specifyAcquisition= True, acquisition = 'Acquisition_1658220161', do_filter = True, upper_limit=0.5,baseline_correct=True,is_solicit = True)
parent7 = ['Run_1658228893.hdf5']
file7 = 'Acquisition_1658228975' # 31.7 Post IV
parent8 = ['Run_1658229202.hdf5']
file8 = 'Acquisition_1658229218' #33.7 post IV
parent9=['Run_1658229455.hdf5']
file9 = 'Acquisition_1658229480' #34.7 Post IV
parent10=['Run_1658230053.hdf5']
file10 = 'Acquisition_1658230076' #35.7 Post IV
parent11 = ['Run_1658230333.hdf5']
file11 = 'Acquisition_1658230344' #32.7 Post IV
filespost = [file7,file8,file9,file10,file11]
parentspost = [parent7,parent8,parent9,parent10,parent11]
runspost = []
for i in range(len(filespost)):
    run_spe_post = RunInfo(parentspost[i],specifyAcquisition=True,acquisition = filespost[i],do_filter = True,upper_limit=upperlim[i],baseline_correct=True,prominence=proms[i])
    runspost.append(run_spe_post)

In [ ]:
#run_spe_solicited = RunInfo(['Run_1658220019.hdf5'], specifyAcquisition= True, acquisition = 'Acquisition_1658220161', do_filter = True, upper_limit=0.5,baseline_correct=True,is_solicit = True)
parent1 = ['Run_1658218686.hdf5']
file1 = 'Acquisition_1658218999' #32.7 V GXe
parent2 = ['Run_1658219411.hdf5']
file2 = 'Acquisition_1658219456' #31.7 V GXe
parent3 = ['Run_1658219616.hdf5']
file3 = 'Acquisition_1658219624' #31.7 V Gxe
parent4 = ['Run_1658220709.hdf5']
file4 = 'Acquisition_1658220750'#34.7 V Gxe
parent5 = ['Run_1658221078.hdf5'] 
file5 = 'Acquisition_1658221117' #33.7 V Gxe
#parent6 = ['Run_1658221468.hdf5']
#file6 = 'Acquisition_1658221483' #35.7 V
files = [file1,file2,file3,file4,file5]
parents = [parent1,parent2,parent3,parent4,parent5]
proms = [.0025,.001,.0025,.005,.005] #.01,.005,.005,.01,.01
upperlims=[0.35,0.35,0.35,0.34,0.35]
runs = []
for i in range(len(files)):
    run_spe = RunInfo(parents[i],specifyAcquisition=True,acquisition = files[i],do_filter = True,upper_limit=upperlims[i],baseline_correct=True,prominence=proms[i])
    runs.append(run_spe)
baises = [run.bias for run in runs]

In [3]:
run_spe_solicited.plot_hists('192','.',new=True)

NameError: name 'runs' is not defined

In [ ]:
RunInfo(['Run_1658220019.hdf5'],do_filter=False,upper_limit = 1,baseline_correct=True,plot_waveforms=True,fourier=False)

In [ ]:
for run in runspost:
    run.plot_hists('192','.',new=True)

In [ ]:
lower=[0.00416,0.00555,0.00587,0.006453,0.00409]
center=[0.00597,0.0091,0.01074,0.01147,0.00779]
high = [lower[i]+4*center[i] for i in range(len(lower))]
biases = [run.bias for run in runspost]
a_guess = 0.01674
b_guess = -0.4598
n=2
T=192
con = 'GXe'
info_spe=MeasurementInfo()
info_spe.condition=con
info_spe.date=runspost[n].date.decode('utf-8')
info_spe.temperature=T
info_spe.bias = biases[n]
info_spe.baseline_numbins=50
info_spe.peaks_numbins=150
info_spe.data_type='h5'
wp=WaveformProcessor(info_spe,run_info_self=runspost[n],run_info_solicit=run_spe_solicited,baseline_correct=True,range_low=lower[n],range_high=high[n],center = center[n])
wp.process(do_spe=True,do_alpha=False)
wp.plot_both_histograms()
wp.plot_peak_histograms()
wp.plot_spe()

In [ ]:
campaign_spe=[]
for i in range(len(lower)):
    if i == 0:
        continue
    T=192
    con='GXe'
    info_spe=MeasurementInfo()
    info_spe.condition=con
    info_spe.date=runspost[i].date.decode('utf-8')
    info_spe.temperature=T
    info_spe.bias = biases[i]
    info_spe.baseline_numbins=50
    info_spe.peaks_numbins=150
    info_spe.data_type='h5'
    wp_spe = WaveformProcessor(info_spe,run_info_self=runspost[i],run_info_solicit=run_spe_solicited,baseline_correct=True,range_low=lower[i],range_high = high[i],center = center[i])
    wp_spe.process(do_spe=True,do_alpha=False)
    campaign_spe.append(wp_spe)

In [ ]:
for i in range(len(campaign_spe)):
    campaign_spe[i].plot_peak_histograms()

In [ ]:
#check CA
for i in range(len(campaign_spe)):
    print('bias: ' + str(biases[i]) + '; ' + str(campaign_spe[i].get_CA()))

In [ ]:
invC_spe_filter = 0.01957644114341406 
invC_spe_err_filter = 8.880858907807588e-05

In [ ]:
curr_campaign=campaign_spe
filtered_spe = SPE_data(curr_campaign,invC_spe_filter,invC_spe_err_filter,filtered =True)
filtered_spe.plot_spe(in_ov=False,absolute=False,out_file=None)
print(filtered_spe.v_bd)
print(filtered_spe.v_bd_err)

In [ ]:
filtered_spe.plot_spe(in_ov = True, absolute = True, out_file = None)

In [ ]:
filtered_spe.plot_CA()

## 193K data

In [ ]:
#193K
parent8 = ['Run_1658251531.hdf5'] #34.7 V
file8 = 'Acquisition_1658251572'
parent9 = ['Run_1658251787.hdf5'] #32.7 V
file9 = 'Acquisition_1658251806'
parent10 = ['Run_1658252031.hdf5'] #33.7 V
file10 = 'Acquisition_1658252056'
parent11 = ['Run_1658252273.hdf5'] #31.7 V
file11 = 'Acquisition_1658252310'
parent12 = ['Run_1658252479.hdf5'] #35.7 V
file12 = 'Acquisition_1658252531' #this last acquisition looks funky
parents193 = [parent8,parent9,parent10,parent11,parent12]
files193 = [file8,file9,file10,file11,file12]

In [ ]:
run_spe_solicited = RunInfo(['Run_1658220019.hdf5'], specifyAcquisition= True, acquisition = 'Acquisition_1658220161', do_filter = True, upper_limit=0.5,baseline_correct=True,is_solicit = True)

In [ ]:
upperlimits193 = [0.3,0.3,0.3,0.3,0.35]

In [ ]:
RunInfo(parents193[4],do_filter=False,upper_limit=upperlimits193[4],baseline_correct=False,plot_waveforms=True,fourier=False)

In [ ]:
proms193 = [0.0018,0.002,0.0033,0.002,0.0035]
runs193=[]
for i in range(len(files193)):
    run_spe_193=RunInfo(parents193[i],specifyAcquisition=True,acquisition=files193[i],do_filter = True,upper_limit=upperlimits193[i],baseline_correct=True,prominence=proms193[i])
    runs193.append(run_spe_193)

In [ ]:
for run in runs193:
    run.plot_hists('193','.',new =True)

In [ ]:
biases = [run.bias for run in runs193]

In [ ]:
rangelows = [0.005679,0.0048,0.00594,0.00353,0.00569]
centers = [0.01,0.0073,0.00918,0.0061,0.01171]
rangehighs = [rangelows[i]+4*centers[i] for i in range(len(rangelows))]
n=1
T = 193
con = 'GXe'
info_spe = MeasurementInfo()
info_spe.condition = con
info_spe.date=runs193[n].date.decode('utf-8')
info_spe.temperature=T
info_spe.bias = biases[n]
info_spe.baseline_numbins = 150
info_spe.peaks_numbins=150
info_spe.data_type='h5'
wp = WaveformProcessor(info_spe,run_info_self=runs193[n],run_info_solicit=run_spe_solicited,baseline_correct=True,range_low=rangelows[n],range_high=rangehighs[n],center = centers[n])
wp.process(do_spe=True,do_alpha=False)
wp.plot_both_histograms()
wp.plot_peak_histograms()
wp.plot_spe()

In [ ]:
campaign_spe=[]
for i in range(len(runs193)):
    if i == 3:
        continue
    if i ==4:
        continue
    T=193
    con = 'GXe'
    info_spe = MeasurementInfo()
    info_spe.condition=con
    info_spe.date=runs193[i].date.decode('utf-8')
    info_spe.temperature = T
    info_spe.bias = biases[i]
    info_spe.baseline_numbins=50
    info_spe.peaks_numbins=150
    info_spe.data_type='h5'
    wp_spe = WaveformProcessor(info_spe,run_info_self=runs193[i],run_info_solicit=run_spe_solicited,baseline_correct=True,range_low=rangelows[i],range_high=rangehighs[i],center=centers[i])
    wp_spe.process(do_spe=True,do_alpha=False)
    wp_spe.plot_spe()
    campaign_spe.append(wp_spe)

In [ ]:
for i in range(len(campaign_spe)):
    campaign_spe[i].plot_peak_histograms()

In [ ]:
#check CA
for i in range(len(campaign_spe)):
    print('bias: ' + str(biases[i]) + '; ' + str(campaign_spe[i].get_CA()))

In [ ]:
invC_spe_filter = 0.01957644114341406 
invC_spe_err_filter = 8.880858907807588e-05

In [ ]:
curr_campaign =campaign_spe
filtered_spe = SPE_data(curr_campaign,invC_spe_filter,invC_spe_err_filter,filtered=True)
filtered_spe.plot_spe(in_ov=False,absolute = False,out_file = None)
print(filtered_spe.v_bd)
print(filtered_spe.v_bd_err)

In [ ]:
filtered_spe.plot_spe(in_ov = True, absolute = True, out_file = None)